In [3]:
from telebot import TeleBot
import textdistance
from parsing_funcs import *

In [4]:
CHAT_ID = '1323215' # The chat ID where this message should be sent to - my chat with bot
TOKEN = '' # The one you got from @BotFather while creating the bot
bot = TeleBot(TOKEN)

In [3]:
housing_faq = parse_housing_faq()
housing_faq.head()

,question,answer
0,How can I apply for accommodation?,You will need to fill out a form in your onlin...
1,Will I sign a formal rental agreement?,"Yes, you will. Each student signs a rental agr..."
2,Can I contact the Student Housing Office manag...,Yes. If you have any questions about your acco...
3,How will I pay for accommodation? Via the univ...,If the contract (rental or accommodation contr...
4,Will there be a security deposit?,All accommodation options require a security d...


In [5]:
programmes = parse_programmes()
programmes.head()

,programme,faculty,link,type,department,location,duration,schedule,language
0,07.00.00 Architecture,Urban Planning,https://www.hse.ru/en/ba/up/,bachelors,Faculty of Urban and Regional Development,Moscow,5 years,Full-time,Russian and English*
1,06.00.00 Biology,Cell and Molecular Biotechnology,https://www.hse.ru/en/ba/cmb/,bachelors,Faculty of Biology and Biotechnology,Moscow,4 years,Full-time,Russian and English*
2,06.00.00 Biology,Cognitive Neurobiology,https://www.hse.ru/en/ba/cmb/,bachelors,Faculty of Biology and Biotechnology,Moscow,4 years,Full-time,Russian and English*
3,58.00.00 Asian and African Studies,Asian and African Studies,https://www.hse.ru/en/ba/oriental/,bachelors,Faculty of World Economy and International Aff...,Moscow,5 years,Full-time,Russian and English*
4,58.00.00 Asian and African Studies,Asian and African Studies,https://www.hse.ru/en/ba/oriental/,bachelors,Saint-Petersburg School of Social Sciences and...,Saint Petersburg,5 years,Full-time,Russian and English*


In [7]:
@bot.message_handler(content_types=['text'])
def get_user_command_choice(message, housing_faq_questions_similarity_barrier=0, programmes_similarity_barrier=0):
    if message.text == "/accomodation":
        bot.send_message(message.from_user.id, "Please send me your question ↓")
        
        # analyzing the question
        bot.register_next_step_handler(
            message, 
            lambda x: get_accomodation_answer(
                x, 
                housing_faq_questions_similarity_barrier=housing_faq_questions_similarity_barrier
            )
        )
    elif message.text == "/costs":
        bot.send_message(message.from_user.id, "Not supprted yet")
    elif message.text == "/ranking":
        bot.send_message(message.from_user.id, "Not supprted yet")
    elif message.text == "/programmes":
        bot.send_message(message.from_user.id, "Which faculty are you interested in?")
        
        # analyzing the question
        bot.register_next_step_handler(
            message,
            lambda x: get_programmes_answer(
                x, 
                programmes_similarity_barrier=programmes_similarity_barrier
            )
        )
    else:
        bot.send_message(message.from_user.id, "Please choose one of the commands in the menu for me to help you!")

In [8]:
def get_sentences_similarity(q1, q2):
    return textdistance.hamming.normalized_similarity(q1, q2)

In [9]:
def get_accomodation_answer(message, housing_faq_questions_similarity_barrier=0): 
    
    # go to the necessary table within the database & fetch most recent date info: housing_faq
    # hosing_faq = <QUERY>
    
    housing_faq['score'] = housing_faq['question'].apply(lambda x: get_sentences_similarity(message.text, x))
    max_score = housing_faq['score'].max()
    
    if max_score < housing_faq_questions_similarity_barrier:
        reply = "Sorry, we do not seem to know the answer. Please contact test@gmail.com"
    else:
        # returning the first answer in case there are multiple
        reply = housing_faq[housing_faq['score'] == max_score]['answer'].values[0]
    
    bot.send_message(message.from_user.id, reply)

In [35]:
def send_pandas_dataframe(df):
    """https://stackoverflow.com/questions/49345960/how-do-i-send-tables-with-telegram-bot-api"""

In [33]:
def get_programmes_answer(message, programmes_similarity_barrier=0):
    
    def get_language_answer(message, faculty):
        """given faculty & language info, return the list of programmes"""
        language_input = message.text.lower().strip().replace(' ', '')
        
        # checking each possible part
        for part, full in zip(['rus', 'eng', 'ger', 'pol', 'STOP'], ['Russian', 'English', 'German', 'Polish', 'STOP']):
            if part in language_input:
                relevant_programmes = programmes[
                    (programmes['faculty'] == faculty)
                    & (programmes['language'].str.contains(full))
                ]
                break # found the language!
            if part == 'STOP':
                bot.send_message(message.from_user.id, "Did not find such language. Please try again.")
                return None # finish the function
        
        bot.send_message(message.from_user.id, "REACHED THE END!")
                
    
    # go to the necessary table within the database & fetch most recent date info: programmes
    # programmes = <QUERY>
    
    programmes['score'] = programmes['faculty'].apply(lambda x: get_sentences_similarity(message.text, x))
    max_score = programmes['score'].max()
    print(max_score)
    
    if max_score < programmes_similarity_barrier:
        reply = "Sorry, couldn't find this faculty. Take a look at 'Urban Planning' maybe :)"
        bot.send_message(message.from_user.id, reply)
    else:
        # the desired faculty
        faculty = programmes[programmes['score'] == max_score]['faculty'].values[0]
        # asking for language
        bot.send_message(message.from_user.id, "Which language do you want to study in?")
        
        # analyzing the question
        bot.register_next_step_handler(
            message,
            lambda x: get_language_answer(
                x, 
                faculty=faculty
            )
        )

In [34]:
bot.polling(none_stop=True, interval=0)

1.0
language_input:  russian
russian:  True
rus Russian russian
